In [2]:
import os
from google.cloud import bigquery

In [3]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "analytics-399003-e59dddbd35b5.json"

In [4]:
client = bigquery.Client()

In [5]:
sql_query = """SELECT * FROM `analytics-399003.analytics_406290682.events_20230918` LIMIT 3"""

In [6]:
job = client.query(sql_query)

In [7]:
for row in job.result():
    print(row)

Row(('20230918', 1695048275080431, 'sec20_timer', [{'key': 'page_title', 'value': {'string_value': 'Viktor Byskov - Viktor Byskov', 'int_value': None, 'float_value': None, 'double_value': None}}, {'key': 'ga_session_id', 'value': {'string_value': None, 'int_value': 1695048254, 'float_value': None, 'double_value': None}}, {'key': 'page_location', 'value': {'string_value': 'https://viktorbyskov.dk/', 'int_value': None, 'float_value': None, 'double_value': None}}, {'key': 'session_engaged', 'value': {'string_value': '1', 'int_value': None, 'float_value': None, 'double_value': None}}, {'key': 'engagement_time_msec', 'value': {'string_value': None, 'int_value': 1591, 'float_value': None, 'double_value': None}}, {'key': 'ga_session_number', 'value': {'string_value': None, 'int_value': 1, 'float_value': None, 'double_value': None}}, {'key': 'engaged_session_event', 'value': {'string_value': None, 'int_value': 1, 'float_value': None, 'double_value': None}}], None, None, 777008367, None, None, 

In [39]:
job = client.get_job('bquxjob_72bb18f5_18ab14e942e') # Job ID inserted based on the query results selected to explore
print(job.query)

SELECT * FROM `analytics-399003.analytics_406290682.events_20230918` LIMIT 1000


In [59]:
dataset = "analytics_406290682"
tablename = "events_20230915"

sql = f"""
SELECT * 
FROM `analytics-399003.{dataset}.{tablename}`
LIMIT 1000
"""

In [60]:
results = client.query(sql).to_dataframe()

In [61]:
results.head()

,event_date,event_timestamp,event_name,event_params,event_previous_timestamp,event_value_in_usd,event_bundle_sequence_id,event_server_timestamp_offset,user_id,user_pseudo_id,...,geo,app_info,traffic_source,stream_id,platform,event_dimensions,ecommerce,items,collected_traffic_source,is_active_user
0,20230915,1694777664810827,click,"[{'key': 'engagement_time_msec', 'value': {'st...",<NA>,NaN,749678411,<NA>,None,203902067.1694777657,...,"{'continent': 'Europe', 'country': 'Denmark', ...",None,"{'name': '(direct)', 'medium': '(none)', 'sour...",6092560544,WEB,None,None,[],None,True
1,20230915,1694777657414419,page_view,"[{'key': 'page_title', 'value': {'string_value...",<NA>,NaN,742282003,<NA>,None,203902067.1694777657,...,"{'continent': 'Europe', 'country': 'Denmark', ...",None,"{'name': '(direct)', 'medium': '(none)', 'sour...",6092560544,WEB,None,None,[],None,True
2,20230915,1694777661244344,scroll_10,"[{'key': 'scroll_depth', 'value': {'string_val...",<NA>,NaN,746111928,<NA>,None,203902067.1694777657,...,"{'continent': 'Europe', 'country': 'Denmark', ...",None,"{'name': '(direct)', 'medium': '(none)', 'sour...",6092560544,WEB,None,None,[],None,True
3,20230915,1694777657414419,first_visit,"[{'key': 'page_title', 'value': {'string_value...",<NA>,NaN,742282003,<NA>,None,203902067.1694777657,...,"{'continent': 'Europe', 'country': 'Denmark', ...",None,"{'name': '(direct)', 'medium': '(none)', 'sour...",6092560544,WEB,None,None,[],None,True
4,20230915,1694777657414419,session_start,"[{'key': 'page_location', 'value': {'string_va...",<NA>,NaN,742282003,<NA>,None,203902067.1694777657,...,"{'continent': 'Europe', 'country': 'Denmark', ...",None,"{'name': '(direct)', 'medium': '(none)', 'sour...",6092560544,WEB,None,None,[],None,True


In [62]:
columns_to_use = ['event_date', 'event_timestamp', 'event_name', 'event_params',
       'event_bundle_sequence_id',
       'user_pseudo_id',
       'user_first_touch_timestamp', 'device', 'geo',
       'traffic_source', 'stream_id', 'platform']

In [63]:
df = results.copy()

In [64]:
df = df[columns_to_use]

In [65]:
df_1 = df.copy()
df_1.head(1)

,event_date,event_timestamp,event_name,event_params,event_bundle_sequence_id,user_pseudo_id,user_first_touch_timestamp,device,geo,traffic_source,stream_id,platform
0,20230915,1694777664810827,click,"[{'key': 'engagement_time_msec', 'value': {'st...",749678411,203902067.1694777657,1694777657414419,"{'category': 'desktop', 'mobile_brand_name': '...","{'continent': 'Europe', 'country': 'Denmark', ...","{'name': '(direct)', 'medium': '(none)', 'sour...",6092560544,WEB


In [66]:
e = "event_params_"
df[e + "page_title"] = None
df[e + "ga_session_id"] = None
df[e + "page_location"] = None
df[e + "session_engaged"] = None
df[e + "engagement_time_msec"] = None
df[e + "ga_session_number"] = None
df[e + "engaged_session_event"] = None

for index, row in df.iterrows():
    eventparams = row["event_params"]
    for param in eventparams:
        if param['key'] == 'page_title':
            df.at[index, "event_params_page_title"] = param['value']['string_value']

        elif param['key'] == 'ga_session_id':
            df.at[index, "event_params_ga_session_id"] = param['value']['int_value']

        elif param['key'] == 'page_location':
            df.at[index, "event_params_page_location"] = param['value']['string_value']

        elif param['key'] == 'session_engaged':
            df.at[index, "event_params_session_engaged"] = param['value']['string_value']

        elif param['key'] == 'engagement_time_msec':
            df.at[index, "event_params_engagement_time_msec"] = param['value']['int_value']

        elif param['key'] == 'ga_session_number':
            df.at[index, "event_params_ga_session_number"] = param['value']['int_value']

        elif param['key'] == 'engaged_session_event':
            df.at[index, "event_params_engaged_session_event"] = param['value']['int_value']

            break

In [67]:
df.drop(columns=["event_params", "device", "geo", "traffic_source"], inplace=True)

In [68]:
df.head(1)

,event_date,event_timestamp,event_name,event_bundle_sequence_id,user_pseudo_id,user_first_touch_timestamp,stream_id,platform,event_params_page_title,event_params_ga_session_id,event_params_page_location,event_params_session_engaged,event_params_engagement_time_msec,event_params_ga_session_number,event_params_engaged_session_event
0,20230915,1694777664810827,click,749678411,203902067.1694777657,1694777657414419,6092560544,WEB,Viktor Byskov - Viktor Byskov,1694777657,https://viktorbyskov.dk/,0,3661,1,None


In [69]:
df.columns

Index(['event_date', 'event_timestamp', 'event_name',
       'event_bundle_sequence_id', 'user_pseudo_id',
       'user_first_touch_timestamp', 'stream_id', 'platform',
       'event_params_page_title', 'event_params_ga_session_id',
       'event_params_page_location', 'event_params_session_engaged',
       'event_params_engagement_time_msec', 'event_params_ga_session_number',
       'event_params_engaged_session_event'],
      dtype='object')

In [35]:
dataset_id = "analytics-399003.analytics_406290682"
tables = client.list_tables(dataset_id)

In [36]:
print("Tables contained in '{}':".format(dataset_id))
for table in tables:
    print("{}.{}.{}".format(table.project, table.dataset_id, table.table_id))

Tables contained in 'analytics-399003.analytics_406290682':
analytics-399003.analytics_406290682.events_20230914
analytics-399003.analytics_406290682.events_20230915
analytics-399003.analytics_406290682.events_20230916
analytics-399003.analytics_406290682.events_20230917
analytics-399003.analytics_406290682.events_20230918
analytics-399003.analytics_406290682.pseudonymous_users_20230914
analytics-399003.analytics_406290682.pseudonymous_users_20230915
analytics-399003.analytics_406290682.pseudonymous_users_20230916
analytics-399003.analytics_406290682.pseudonymous_users_20230917
analytics-399003.analytics_406290682.pseudonymous_users_20230918


In [58]:
df.to_csv("cleaned_table.csv")